# Plotting SSM/I data from a NetCDF

In [1]:
import xarray as xr
from datetime import date
import matplotlib.pyplot as plt
from matplotlib import colors
from pathlib import Path

# Test should run from root
import os

os.chdir("..")

In [2]:
from omegaconf import OmegaConf
from multiearth.api import extract_assets
from multiearth.config import ConfigSchema

cfg = OmegaConf.load("config/nsidc-ssmi.yaml")
cfg_schema = OmegaConf.structured(ConfigSchema)
cfg = OmegaConf.merge(cfg_schema, cfg)

# Constrain the test to only pull one item and one band
for pvdr in cfg.providers:
    for coll in pvdr.collections:
        coll.max_items = 1

success = extract_assets(cfg)
print("Successfully extracted assets." if success else "Asset extraction failed.")

15:20:40 INFO - Extraction details for provider ProviderKey.EARTHDATA with collection NSIDC-0530: 
id: NSIDC-0530
assets:
- all
outdir: null
datetime: null
aoi_file: null
max_items: 1

15:20:47 INFO - EarthData Provider returned 1 items for NSIDC-0530 for datetime 1999-01-01/2012-12-31
15:20:47 INFO - 2 assets did not specify file size, will query size directly with http request (this may take a few moments)
system.query_asset_sizes=False can be used to disable this behavior


  0%|          | 0/2 [00:00<?, ?it/s]

15:20:49 INFO - 1 assets have unknown file size
15:20:49 INFO - 

****************************************************************************************************
To Extract:
EarthData Provider: NSIDC-0530.v1


Collection               | Key                 | Description
--------------------------------------------------------------------------------
NSIDC-0530.v1            | data                | 
NSIDC-0530.v1            | metadata            | 


Collection size                     1 MB
Size of remaining data to download  0 MB
Number of assets with unknown size: 1

****************************************************************************************************



Successfully extracted assets.


Let's load our NetCDF file we obtained from MultiEarth

In [3]:
# Find the correct NC file and load it
# We avoid hardcoding the path here in case the API changes down the road

fpath = Path(cfg["default_collection"]["outdir"])
collection_for_nb = "NSIDC-0530"
nc_files = list((fpath / collection_for_nb).glob("**/*.nc"))

# We know there is only one file pulled, assert
assert (
    len(nc_files) == 1
), f"There were {len(nc_files)} files pulled. `max_items: 1` failed to constrain."

nc_file_fpath = nc_files[0]
ncfile = xr.open_dataset(nc_file_fpath)

In [4]:
ncfile

<xarray.Dataset>
Dimensions:                                         (time: 1, rows: 720,
                                                     cols: 720)
Coordinates:
  * time                                            (time) datetime64[ns] 199...
    latitude                                        (rows, cols) float32 ...
    longitude                                       (rows, cols) float32 ...
  * rows                                            (rows) int32 8987500 ... ...
  * cols                                            (cols) int32 -8987500 ......
Data variables:
    merged_snow_cover_extent                        (time, rows, cols) float32 ...
    ims_snow_cover_extent                           (time, rows, cols) float32 ...
    passive_microwave_gap_filled_snow_cover_extent  (time, rows, cols) float32 ...
    modis_cloud_gap_filled_snow_cover_extent        (time, rows, cols) float32 ...
    coord_system                                    |S1 ...
Attributes: (12/26)
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0, GDS v2.0
    standard_name_vocabulary:  CF Standard Name Table (v22, 12 February 2013)
    id:                        nhtsd25e2_19990101_v01r01.nc
    naming_authority:          gov.nasa.eosdis
    reference:                 http://dx.doi.org/10.5067/MEASURES/CRYOSPHERE/...
    ...                        ...
    geospatial_lat_min:        0
    geospatial_lat_max:        90
    geospatial_lon_min:        -180
    geospatial_lon_max:        180
    spatial_resolution:        25 km
    license:                   No restrictions on access or use

In [5]:
ncfile_select = ncfile.sel(time="1999-01-01T00:00:00.000000000")
ncfile_select

<xarray.Dataset>
Dimensions:                                         (rows: 720, cols: 720)
Coordinates:
    time                                            datetime64[ns] 1999-01-01
    latitude                                        (rows, cols) float32 ...
    longitude                                       (rows, cols) float32 ...
  * rows                                            (rows) int32 8987500 ... ...
  * cols                                            (cols) int32 -8987500 ......
Data variables:
    merged_snow_cover_extent                        (rows, cols) float32 ...
    ims_snow_cover_extent                           (rows, cols) float32 ...
    passive_microwave_gap_filled_snow_cover_extent  (rows, cols) float32 ...
    modis_cloud_gap_filled_snow_cover_extent        (rows, cols) float32 ...
    coord_system                                    |S1 b''
Attributes: (12/26)
    Conventions:               CF-1.6
    Metadata_Conventions:      CF-1.6, Unidata Dataset Discovery v1.0, GDS v2.0
    standard_name_vocabulary:  CF Standard Name Table (v22, 12 February 2013)
    id:                        nhtsd25e2_19990101_v01r01.nc
    naming_authority:          gov.nasa.eosdis
    reference:                 http://dx.doi.org/10.5067/MEASURES/CRYOSPHERE/...
    ...                        ...
    geospatial_lat_min:        0
    geospatial_lat_max:        90
    geospatial_lon_min:        -180
    geospatial_lon_max:        180
    spatial_resolution:        25 km
    license:                   No restrictions on access or use